In [27]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
# Chargement des données
base_path = './Data/Data_utiles/Data_ML/'

# Liste pour stocker chaque DataFrame
dataframes = []

# Boucle pour lire chaque fichier CSV de 1995 à 2018
for year in range(1995, 2019):
    file_path = f'{base_path}infos_joueurs_{year}.csv'
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concaténer tous les DataFrames en un seul
data = pd.concat(dataframes, ignore_index=True)

columns = [
    "name", "hand", "height", "rang", "matchs", "win", 
    "pourc_return_win_pnt", "pourc_break_games", "pourc_break_point_made", 
    "pourc_break_point_saved", "pourc_serv_games_win", "pourc_serv_in", 
    "mean_ranking_oppo", "pourc_serv_win_pnt", "Return Rating", 
    " % Serve Return Points Won", " % 2nd Serve Return Points Won", 
    " % Return Games Won", " % Break Points Converted", "Under Pressure Rating", 
    " % Break Point Saved", " % Break Points Converted Pressure", 
    " % Deciding Sets Won", " % Tie Breaks Won"
]
 # Complétez avec les autres noms de colonnes
df = pd.DataFrame(data, columns=columns)
df['hand'] = df['hand'].map({'R': 1, 'L': 0})

# Vérifier les modifications
print(df['hand'].head())
print(df.columns)
# Mise à jour des caractéristiques en excluant les colonnes spécifiées


# Séparation des caractéristiques et des étiquettes
features = df.drop(['name','rang','Under Pressure Rating', ' % Break Point Saved', 
    ' % Break Points Converted Pressure', ' % Deciding Sets Won', 
    ' % Tie Breaks Won'], axis=1)
labels = df['rang']
  # Affiche les premières lignes pour vérifier les données
indices_with_nan = features[features.isna().any(axis=1)].index

# Supprimer ces lignes de features
features_clean = features.dropna()

# Supprimer les mêmes lignes de labels
labels_clean = labels.drop(indices_with_nan) # Affiche la taille de l'ensemble des étiquettes
print(features_clean.shape)  # Doit montrer (N, nombre_de_colonnes)
print(labels_clean.shape) 
# Prétraitement : Encodage one-hot pour les variables catégorielles et normalisation pour les numériques
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['hand']),
        ('num', StandardScaler(), features.columns.drop('hand'))
    ])

# Pipeline : Prétraitement + modèle
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])
# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features_clean, labels_clean, test_size=0.2, random_state=42)

# Vérification et suppression des NaN dans les ensembles d'entraînement et de test
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Assurez-vous que les indices correspondent
X_test = X_test.dropna()
y_test = y_test[X_test.index]

# Entraînement du modèle
model.fit(X_train, y_train)

# Évaluation du modèle
y_pred = model.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

#réalisation de cross validation
model = RandomForestRegressor()

# Utilisation de KFold pour un contrôle plus précis
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Évaluation du modèle en utilisant la cross-validation avec le pipeline
scores = cross_val_score(model, features_clean, labels_clean, cv=kf, scoring='neg_mean_squared_error')


rmse_scores = np.sqrt(-scores)
print("RMSE par fold : ", rmse_scores)
print("RMSE moyen : ", rmse_scores.mean())


0    1
1    1
2    1
3    1
4    1
Name: hand, dtype: int64
Index(['name', 'hand', 'height', 'rang', 'matchs', 'win',
       'pourc_return_win_pnt', 'pourc_break_games', 'pourc_break_point_made',
       'pourc_break_point_saved', 'pourc_serv_games_win', 'pourc_serv_in',
       'mean_ranking_oppo', 'pourc_serv_win_pnt', 'Return Rating',
       ' % Serve Return Points Won', ' % 2nd Serve Return Points Won',
       ' % Return Games Won', ' % Break Points Converted',
       'Under Pressure Rating', ' % Break Point Saved',
       ' % Break Points Converted Pressure', ' % Deciding Sets Won',
       ' % Tie Breaks Won'],
      dtype='object')
(639, 17)
(639,)
RMSE: 14.807470586033592
RMSE par fold :  [14.86246462 12.8077192  13.35595452 13.64122798 15.48710602]
RMSE moyen :  14.030894467351866


In [33]:
print(features_clean)

      hand  height  matchs   win  pourc_return_win_pnt  pourc_break_games  \
0        1   196.0    62.0  39.0              0.361919           0.207784   
1        1   190.0    83.0  62.0              0.425043           0.348266   
2        1   188.0    42.0  18.0              0.346259           0.179832   
3        1   178.0    53.0  32.0              0.388201           0.245503   
4        1   193.0    51.0  28.0              0.377143           0.236942   
...    ...     ...     ...   ...                   ...                ...   
3123     1   196.0    37.0  18.0              0.336723           0.155910   
3124     0   188.0    35.0  18.0              0.382003           0.266190   
3125     1   183.0    50.0  28.0              0.383998           0.245907   
3128     1   178.0    19.0   7.0              0.331022           0.165849   
3129     1   196.0    14.0   4.0              0.334671           0.147740   

      pourc_break_point_made  pourc_break_point_saved  pourc_serv_games_win

Deuxième modèle de ML :GradientBoostingRegressor
GradientBoostingRegressor est un modèle puissant qui construit des arbres de décision de manière séquentielle, où chaque arbre tente de corriger les erreurs des arbres précédents. Voici un exemple de code utilisant GradientBoostingRegressor 

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Prétraitement : Encodage one-hot pour les variables catégorielles et normalisation pour les numériques
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['hand']),  # Assurez-vous que 'hand' est la colonne catégorielle
        ('num', StandardScaler(), features_clean.select_dtypes(include=['float64', 'int64']).columns)
    ])

# Création du pipeline avec GradientBoostingRegressor
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features_clean, labels_clean, test_size=0.2, random_state=42)

# Entraînement du modèle
model.fit(X_train, y_train)

# Évaluation du modèle sur l'ensemble de test
y_pred = model.predict(X_test)
print("RMSE sur l'ensemble de test :", np.sqrt(mean_squared_error(y_test, y_pred)))

# Réalisation de la cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, features_clean, labels_clean, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)
print("RMSE par fold :", rmse_scores)
print("RMSE moyen :", rmse_scores.mean())

RMSE sur l'ensemble de test : 13.83982480044501
RMSE par fold : [13.8398248  13.25627024 13.39855893 14.40236881 14.58997891]
RMSE moyen : 13.89740033870961


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Création du pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Paramètres pour la recherche en grille
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 4, 5]
}

# Recherche en grille avec validation croisée
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(features_clean, labels_clean)

# Meilleur modèle
best_model = grid_search.best_estimator_

# Afficher les meilleurs paramètres
print("Meilleurs paramètres : ", grid_search.best_params_)

# Évaluation du modèle
X_train, X_test, y_train, y_test = train_test_split(features_clean, labels_clean, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE sur l'ensemble de test :", rmse)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres :  {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 300}
RMSE sur l'ensemble de test : 13.926325636250269


In [8]:
import pandas as pd

# Exemple de données
data = pd.read_csv('./Data/Data_utiles/info_rank.csv')['ranking_date']
df = pd.DataFrame(data)

# Convertir les dates en format datetime



df['ranking_date'] = pd.to_datetime(df['ranking_date'], format='%Y%m%d')

# Trier les dates
df = df.sort_values(by='ranking_date')

# Extraire la dernière date de chaque année
last_dates = df.groupby(df['ranking_date'].dt.year)['ranking_date'].last()

# Convertir les dates dans le format d'origine (AAAAMMJJ)
formatted_dates = [date.strftime('%Y%m%d') for date in last_dates]

print(formatted_dates)

['19901231', '19911230', '19921228', '19931227', '19941226', '19951225', '19961230', '19971229', '19981228', '19991227', '20001225', '20011231', '20021230', '20031229', '20041227', '20051226', '20061225', '20071231', '20081229', '20091228', '20101227', '20111226', '20121231', '20131230', '20141229', '20151228', '20161226', '20171225', '20181231', '20191230', '20201228', '20211227', '20221226']
